In [2]:
import pandas as pd
import os

def make_dir(p):
    if not os.path.exists(p):
        os.makedirs(p)

## 2-hop

In [3]:
SOL_DATA_PATH = os.path.join('DCNDP_sols')

dcndp_2hop_agg_df = pd.read_csv('DCNDP_sols/DCNDP_NL_D2_2023_06_14_healthAuthority_ID/day2_0/hybrid_rate_0/partition_results.csv')
dcndp_2hop_disagg_df = pd.read_csv('DCNDP_sols/DCNDP_NL_D2_2023_06_14_healthAuthority_ID_disagg/day2_0/hybrid_rate_0/partition_results.csv')


In [4]:
# merge the two dataframes based on healthAuthority_ID,	partition,	c_count, add agg and disagg suffixes
dcndp_2hop_df = pd.merge(dcndp_2hop_agg_df, dcndp_2hop_disagg_df, on=['healthAuthority_ID', 'partition', 'c_count'], suffixes=('_agg', '_disagg'))

dcndp_2hop_df.columns


cols_to_keep = ['healthAuthority_ID', 'partition', 'c_count', 'component_size_agg','component_edges_agg', 'budget_agg',\
                'pre_optimization_short_pairs_agg', 'num_simplicials_agg', 'm_status_agg','m_obj_agg', 'm_MIPGap_agg', \
                'm_runtime_agg','m_status_disagg', 'm_obj_disagg', 'm_MIPGap_disagg', 'm_runtime_disagg']

dcndp_2hop_df = dcndp_2hop_df[cols_to_keep]

dcndp_2hop_df['density'] = 100 * (dcndp_2hop_df['component_edges_agg']/((dcndp_2hop_df['component_size_agg']*(dcndp_2hop_df['component_size_agg'] - 1))/2))

new_col_names = ['healthAuthority_ID', 'partition', 'c_count', 'component_size', 'component_edges', 'budget',\
                 'original_objective', 'num_simplicials', 'm_status_agg', 'm_obj_agg', 'm_MIPGap_agg', \
                'm_runtime_agg', 'm_status_disagg', 'm_obj_disagg', 'm_MIPGap_disagg', 'm_runtime_disagg', 'density']

dcndp_2hop_df.columns = new_col_names

dcndp_2hop_df['m_MIPGap_agg'] = 100 * dcndp_2hop_df['m_MIPGap_agg']
dcndp_2hop_df['m_MIPGap_disagg'] = 100 * dcndp_2hop_df['m_MIPGap_disagg']

dcndp_2hop_df['perc_reduction_obj_agg'] = 100 * (dcndp_2hop_df['original_objective'] - dcndp_2hop_df['m_obj_agg'])/dcndp_2hop_df['original_objective']
dcndp_2hop_df['perc_reduction_obj_disagg'] = 100 * (dcndp_2hop_df['original_objective'] - dcndp_2hop_df['m_obj_disagg'])/dcndp_2hop_df['original_objective']

In [5]:
summary_results_path = os.path.join("summary_results")
make_dir(summary_results_path)

dcndp_2hop_df.to_csv(os.path.join(summary_results_path, 'dcndp_2hop_summary_results.csv'), index=False)

In [6]:
#select where density is greater than 0.4
#dcndp_2hop_df = dcndp_2hop_df[dcndp_2hop_df['density'] > 0.5]


## 1-hop

In [8]:
dcndp_1hop_df = pd.read_csv('DCNDP_sols/DCNDP_1hop_NL_D2_2023_06_28_healthAuthority_ID/day2_0/hybrid_rate_0/partition_results.csv')


cols_to_keep = ['healthAuthority_ID', 'partition', 'c_count', 'component_size',\
                'component_edges', 'budget', 'pre_optimization_short_pairs', \
                    'm_status', 'm_obj', 'm_MIPGap', 'm_runtime']

dcndp_1hop_df = dcndp_1hop_df[cols_to_keep]

new_col_names = ['healthAuthority_ID', 'partition', 'c_count', 'component_size',\
                'component_edges', 'budget', 'original_objective', \
                    'm_status', 'm_obj', 'm_MIPGap', 'm_runtime']

dcndp_1hop_df.columns = new_col_names

dcndp_1hop_df['density'] = 100 * (dcndp_1hop_df['component_edges']/((dcndp_1hop_df['component_size']*(dcndp_1hop_df['component_size'] - 1))/2))
dcndp_1hop_df['m_MIPGap'] = 100 * dcndp_1hop_df['m_MIPGap']

dcndp_1hop_df['perc_reduction_obj'] = 100 * (dcndp_1hop_df['original_objective'] - dcndp_1hop_df['m_obj'])/dcndp_1hop_df['original_objective']

dcndp_1hop_df.to_csv(os.path.join(summary_results_path, 'dcndp_1hop_summary_results.csv'), index=False)